In [ ]:
import sys
#Fix problem with ros python path
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')

In [ ]:
import numpy as np
import cv2
import os
import pascal_voc_writer
from tqdm import tqdm

In [ ]:
path = "/media/data/LocalizationData/"
out_path = path+"Output/Validation/CoptersFlying5/"

In [ ]:
os.makedirs(out_path+"Automatic", exist_ok=True)
os.makedirs(out_path+"Automatic_Debug", exist_ok=True)
os.makedirs(out_path+"Manual", exist_ok=True)

def saveGroundtruth(frame, contours, frameNum, size_min, size_max, length_tol, padding=0):
    objCnt = 0
    frameStr = "frame"+str(frameNum).zfill(6)
    writer = pascal_voc_writer.Writer(out_path+frameStr+".png", 1600, 1200)
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        x = x+padding; y = y+padding; w = w-2*padding; h = h-2*padding
        if w > size_min and w < size_max and h > size_min and h < size_max and abs(h-w)/(w+h) < length_tol and y < 1000 and x < 1300:
            if PREPARATION_MODE: cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
            writer.addObject('copter', x, y, x+w, y+h)
            roi = frame[y:y+h, x:x+w]
            objCnt = objCnt+1
    if PREPARATION_MODE:
        cv2.drawContours(frame,contours,-1,(0,255,0),3)
        cv2.imshow('orig',frame)
    else:
        if objCnt == 1:
            cv2.imwrite(out_path+"Automatic/"+frameStr+".jpg", frame)
            cv2.imwrite(out_path+"Automatic_Debug/"+frameStr+".jpg", roi)
            writer.save(out_path+"Automatic/"+frameStr+".xml")
        else:
            cv2.imwrite(out_path+"Manual/"+frameStr+".jpg", frame)
            writer.save(out_path+"Manual/"+frameStr+".xml")

In [ ]:
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()

def generateMaskMOG(frame):
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(40,40))
    kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(20,20))
    mask = fgbg.apply(frame)
    mask = cv2.dilate(mask,kernel1,iterations = 1)
    mask = cv2.erode(mask,kernel2,iterations = 1)
    ret, mask = cv2.threshold(mask, 50, 255, cv2.THRESH_BINARY)
    return mask

def generateMaskAbsDiff(background,frame):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23))
    fgmask = cv2.absdiff(background,frame)
    fgmask_gray = cv2.cvtColor(fgmask, cv2.COLOR_RGB2GRAY)
    ret, mask = cv2.threshold(fgmask_gray, 30, 255, cv2.THRESH_BINARY)
    mask_filtered = mask
    mask_filtered = cv2.dilate(mask_filtered,kernel,iterations = 1)
    #mask_filtered = cv2.morphologyEx(mask_filtered, cv2.MORPH_OPEN, kernel)
    mask_filtered = cv2.morphologyEx(mask_filtered, cv2.MORPH_CLOSE, kernel)

    im2, contours, hierarchy = cv2.findContours(mask_filtered,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    mask_contours = np.zeros(mask_filtered.shape, np.uint8)
    cv2.drawContours(mask_contours, contours, -1, (255,255,255),cv2.FILLED)
    
    #cv2.imshow('mask',mask_filtered)
    return mask_contours

**rember to init subtractor with video first**

In [ ]:
PREPARATION_MODE = True

## Init every run
#fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()

#cap = cv2.VideoCapture(path+'Videos/CopterWithLEDFlight/background6.avi')
#for i in range(30):
#    ret,background = cap.read()
#cap.release()

cap2 = cv2.VideoCapture(path+'Videos/CopterWithLEDFlight/copters_flying5.avi')

for i in tqdm(range(int(cap2.get(cv2.CAP_PROP_FRAME_COUNT)))):
    ret, frame = cap2.read()
    if ret == False:
        break
    if i%4==0:
        mask = generateMaskMOG(frame)
        #mask = generateMaskAbsDiff(background,frame)
        im2, contours, hierarchy = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)    
        saveGroundtruth(frame, contours, i, 50, 300, 0.2, 8)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap2.release()
cv2.destroyAllWindows()